In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Dummies.csv')
data.head()

KeyboardInterrupt: 

In [ ]:
targets = ['Tué', 'Indemne', 'Blessé léger', 'Blessé hospitalisé']
X = data.drop(targets, axis=1)
X = sm.add_constant(X)
Y = [data[target] for target in targets]

C:\Users\Erwan\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
from tqdm import tqdm
X_opts = []
significance_level = .01
for i in tqdm(range(4), desc='Régressions'):
    X_opt = X.copy()
    highest_pvalue = 1
    while X_opt.columns.tolist() and highest_pvalue > significance_level:
        ols = sm.OLS(endog=Y[i], exog=X_opt).fit()
        highest_pvalue = ols.pvalues.max()
        less_significant = ols.pvalues.index[ols.pvalues.argmax()]
        if highest_pvalue > significance_level:
            X_opt.drop(inplace=True, columns=less_significant)
    X_opts.append(X_opt.columns)

Régressions: 100%|██████████| 4/4 [22:43<00:00, 340.98s/it]


In [ ]:
[x.tolist() for x in X_opts]

[['Passager central (à gauche)',
  'Passager central (à droite)',
  'Conducteur',
  'Passager',
  'Piéton',
  'Homme',
  'Femme',
  'Ceinture',
  'Casque',
  'Dispositif enfant',
  'Bicyclette',
  'VL seul',
  'VU seul',
  'Motocyclette >125cm3',
  'VAE',
  'Obstacle : Arbre',
  'Obstacle : Glissière métallique',
  'Obstacle : Bâtiment, mur, pile de pont',
  'Obstacle : Poteau',
  'Obstacle : Parapet',
  'Obstacle : Fossé, talus, paroi rocheuse',
  'Obstacle : Autre obstacle fixe hors chaussée',
  "Obstacle : Buse, tête d'aqueduc",
  'Animal domestique',
  "Choc à l'avant droit",
  "Choc à l'avant gauche",
  "Choc à l'arrière gauche",
  'Sans changement de direction',
  'Même sens, même file',
  'Entre 2 files',
  'À contresens',
  'En se déportant à gauche',
  'En tournant à gauche',
  'En tournant à droite',
  "Pendant une manoeuvre d'évitement",
  'Moteur à hydrocarbures',
  'Autre propulsion',
  'Plein jour',
  'Crépuscule ou aube',
  'Nuit avec éclairage public allumé',
  'En aggl